In [ ]:
pip install tensorflow numpy mnist

Import section


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Portfolio 5/Corrosion

/content/drive/My Drive/Portfolio 5/Corrosion


In [ ]:
import os
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
!unzip Corrosion.zip -d "/content/drive/My Drive/Portfolio 5"



unzip:  cannot find or open Corrosion.zip, Corrosion.zip.zip or Corrosion.zip.ZIP.


**Data Preparation**

In [ ]:
data_dir ='/content/drive/My Drive/Portfolio 5/Corrosion/Corrosion'

rust_images = [os.path.join(data_dir, 'rust', f) for f in os.listdir(os.path.join(data_dir, 'rust'))]
no_rust_images = [os.path.join(data_dir, 'no_rust', f) for f in os.listdir(os.path.join(data_dir, 'no_rust'))]


# Splitting images into training and test sets (10 rust, 10 no rust for test set)
rust_train, rust_test = train_test_split(rust_images, test_size=10, random_state=42)
no_rust_train, no_rust_test = train_test_split(no_rust_images, test_size=10, random_state=42)

train_set = rust_train + no_rust_train
test_set = rust_test + no_rust_test

# Create directories to store train/test sets
os.makedirs('train/rust', exist_ok=True)
os.makedirs('train/no_rust', exist_ok=True)
os.makedirs('test/rust', exist_ok=True)
os.makedirs('test/no_rust', exist_ok=True)

# Move images to appropriate directories
for img in rust_train:
    shutil.copy(img, 'train/rust/')
for img in no_rust_train:
    shutil.copy(img, 'train/no_rust/')
for img in rust_test:
    shutil.copy(img, 'test/rust/')
for img in no_rust_test:
    shutil.copy(img, 'test/no_rust/')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create training and testing datasets
train_data = train_datagen.flow_from_directory(
    'train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    'test/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

Found 752 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


**Task 1**

**CNN Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_simple_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate and train the model
simple_cnn = create_simple_cnn()
history = simple_cnn.fit(train_data, epochs=10, validation_data=test_data)

# Save the model
simple_cnn.save('simple_cnn_model.h5')

# Evaluate the model
test_loss, test_acc = simple_cnn.evaluate(test_data)
print(f"Simple CNN Test Accuracy: {test_acc}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.6013 - loss: 0.6900 - val_accuracy: 0.4500 - val_loss: 0.7302
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7078 - loss: 0.5855 - val_accuracy: 0.6000 - val_loss: 0.5858
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.7846 - loss: 0.4839 - val_accuracy: 0.6000 - val_loss: 0.6923
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8252 - loss: 0.4212 - val_accuracy: 0.7000 - val_loss: 0.6809
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.7882 - loss: 0.4852 - val_accuracy: 0.8000 - val_loss: 0.4389
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8388 - loss: 0.3818 - val_accuracy: 0.7500 - val_loss: 0.4474
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8509 - loss: 0.3722 - val_accuracy: 0.8000 - val_loss: 0.4956
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8559 - loss: 0.3667 - val_accuracy: 0.8000 - val_loss: 0.3923
Epo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.8000 - loss: 0.3870
Simple CNN Test Accuracy: 0.800000011920929


**ResNet50 Model**

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the pre-trained ResNet50 model with ImageNet weights, exclude the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of ResNet50 base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
resnet50_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
resnet50_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_resnet = resnet50_model.fit(train_data, epochs=10, validation_data=test_data)

# Save the model
resnet50_model.save('resnet50_model.h5')

# Evaluate the ResNet50 model
test_loss_resnet, test_acc_resnet = resnet50_model.evaluate(test_data)
print(f"ResNet50 Test Accuracy: {test_acc_resnet}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.5315 - loss: 0.7500 - val_accuracy: 0.5000 - val_loss: 0.7222
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.5052 - loss: 0.7365 - val_accuracy: 0.5000 - val_loss: 0.6972
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.5570 - loss: 0.6912 - val_accuracy: 0.5000 - val_loss: 0.7348
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.5032 - loss: 0.7215 - val_accuracy: 0.5000 - val_loss: 0.6918
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.5638 - loss: 0.6879 - val_accuracy: 0.7000 - val_loss: 0.6891
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.4963 - loss: 0.6977 - val_accuracy: 0.5000 - val_loss: 0.6977
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.5605 - loss: 0.6895 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.486

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6909
ResNet50 Test Accuracy: 0.5


 **Task 2**

In [60]:
# Import necessary libraries
import os
import random
import shutil
import labelme2coco
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import tensorflow as tf
import numpy as np
import imgaug  # for data augmentation
import cv2  # for drawing masks



In [19]:
labelme_folder = '/content/drive/My Drive/Portfolio 5/log-labelled/log-labelled'

In [20]:
# List all files in the directory
all_files = os.listdir(labelme_folder)

In [22]:
image_files = [f for f in all_files if f.endswith(('.jpg', '.png'))]
annotation_files = [f for f in all_files if f.endswith('.json')]

In [23]:
assert len(image_files) == len(annotation_files), "Number of images and annotations don't match!"


In [24]:
# Randomly select 10 images for the test set
test_images = random.sample(image_files, 10)

# Create train and test directories
train_image_dir = os.path.join('train', 'images')
train_annotation_dir = os.path.join('train', 'annotations')
test_image_dir = os.path.join('test', 'images')
test_annotation_dir = os.path.join('test', 'annotations')

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_annotation_dir, exist_ok=True)
os.makedirs(test_image_dir, exist_ok=True)
os.makedirs(test_annotation_dir, exist_ok=True)

In [25]:
# Function to get annotation file corresponding to an image
def get_annotation_file(image_file):
    annotation_file = image_file.rsplit('.', 1)[0] + '.json'
    if annotation_file in annotation_files:
        return annotation_file
    else:
        raise FileNotFoundError(f"Annotation file for image '{image_file}' not found.")

In [26]:
# Move test images and their corresponding annotations
for img in test_images:
    try:
        annotation = get_annotation_file(img)
        shutil.copy(os.path.join(labelme_folder, img), test_image_dir)
        shutil.copy(os.path.join(labelme_folder, annotation), test_annotation_dir)
    except FileNotFoundError as e:
        print(e)

# Move the remaining images and their corresponding annotations to the train set
train_images = [img for img in image_files if img not in test_images]
for img in train_images:
    try:
        annotation = get_annotation_file(img)
        shutil.copy(os.path.join(labelme_folder, img), train_image_dir)
        shutil.copy(os.path.join(labelme_folder, annotation), train_annotation_dir)
    except FileNotFoundError as e:
        print(e)

In [27]:
# Convert Labelme annotations to COCO format
coco_output_train = "train/annotations_coco"
coco_output_test = "test/annotations_coco"

labelme2coco.convert(train_annotation_dir, coco_output_train)
labelme2coco.convert(test_annotation_dir, coco_output_test)

There are 590 listed files in folder annotations.


Converting labelme annotations to COCO format: 100%|██████████| 590/590 [00:06<00:00, 87.71it/s] 


There are 10 listed files in folder annotations.


Converting labelme annotations to COCO format: 100%|██████████| 10/10 [00:00<00:00, 46.80it/s]


In [45]:
# Mask RCNN Configuration
class LogConfig(Config):
    NAME = "log"
    NUM_CLASSES = 1 + 1  # Background + log
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    DETECTION_MIN_CONFIDENCE = 0.9
    BACKBONE = "resnet50"
    LEARNING_RATE = 0.001
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512


In [69]:
# Load Dataset
class LogDataset(utils.Dataset):
    def load_logs(self, dataset_dir, subset):
        self.add_class("log", 1, "log")

        # Add images
        annotations_file = os.path.join(dataset_dir, "annotations_coco.json")
        annotations = labelme2coco.load_json(annotations_file)
        for annotation in annotations["images"]:
            image_path = os.path.join(dataset_dir, "images", annotation["file_name"])
            image_id = annotation["id"]
            self.add_image("log", image_id=image_id, path=image_path, width=annotation["width"], height=annotation["height"])

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        if info["source"] != "log":
            return super(self.__class__, self).load_mask(image_id)

        # Load mask annotations for the image
        annotations = labelme2coco.load_json(os.path.join(os.path.dirname(info["path"]), "annotations_coco.json"))
        masks = []
        for annotation in annotations["annotations"]:
            if annotation["image_id"] == info["id"]:
                mask = np.zeros([info["height"], info["width"]], dtype=np.uint8)
                for seg in annotation["segmentation"]:
                    polygon = np.array(seg).reshape((-1, 2))
                    cv2.fillPoly(mask, [polygon], 1)
                masks.append(mask)

        if len(masks) == 0:
            return super(self.__class__, self).load_mask(image_id)

        masks = np.stack(masks, axis=-1)
        class_ids = np.array([1] * masks.shape[-1], dtype=np.int32)
        return masks, class_ids


In [73]:
# Training
config = LogConfig()

# Fix the issue by using a functional approach to create the Mask RCNN model
COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"
model = modellib.MaskRCNN(mode="training", config=config, model_dir="logs")

model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

# Prepare dataset
dataset_train = LogDataset()
dataset_train.load_logs('train', "train")
dataset_train.prepare()

dataset_val = LogDataset()
dataset_val.load_logs('test', "test")
dataset_val.prepare()

# Train the model
with tf.device("/gpu:0"):
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads',
                augmentation=imgaug.augmenters.Fliplr(0.5))

# Testing the model
model_inference = modellib.MaskRCNN(mode="inference", config=config, model_dir="logs")
model_inference.load_weights(COCO_WEIGHTS_PATH, by_name=True)

# Test and visualize results on the test set
for image_file in os.listdir(test_image_dir):
    if image_file.endswith(('.jpg', '.png')):
        image = cv2.imread(os.path.join(test_image_dir, image_file))
        results = model_inference.detect([image], verbose=1)
        r = results[0]

        # Draw the detected objects on the image
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                    dataset_val.class_names, r['scores'], show_bbox=True, show_mask=True)

        # Save the output image with detections
        output_path = os.path.join('output', image_file)
        os.makedirs('output', exist_ok=True)
        cv2.imwrite(output_path, image)

        # Count the number of detected logs
        num_logs = len(r['class_ids'])
        print(f"Number of detected logs in {image_file}: {num_logs}")

NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, None, 4), dtype=float32, sparse=None, name=input_gt_boxes>',)
  • kwargs={'mask': 'None'}